# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [11]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}' #From testing sandbox 
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2280892717329,
 'week52high': 146.57,
 'week52low': 74.17,
 'week52highSplitAdjustOnly': 150.85,
 'week52lowSplitAdjustOnly': 76.12,
 'week52change': 0.8357017197208984,
 'sharesOutstanding': 16979690587,
 'float': 0,
 'avg10Volume': 97137068,
 'avg30Volume': 90571525,
 'day200MovingAvg': 129.73,
 'day50MovingAvg': 134.61,
 'employees': 149446,
 'ttmEPS': 4.62,
 'ttmDividendRate': 0.8463181524537415,
 'dividendYield': 0.00646071615705396,
 'nextDividendDate': '2021-05-04',
 'exDividendDate': '2021-02-03',
 'nextEarningsDate': '2021-07-15',
 'peRatio': 30.277443543204427,
 'beta': 1.360627933021214,
 'maxChangePercent': 51.70980632151074,
 'year5ChangePercent': 5.148054980395115,
 'year2ChangePercent': 1.601011475027122,
 'year1ChangePercent': 0.8674224701075596,
 'ytdChangePercent': 0.0003678734729045834,
 'month6ChangePercent': 0.20519930836243389,
 'month3ChangePercent': -0.00917894052519514,
 'month1ChangePercent': 0.07922771118820736,
 'da

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [12]:
data['year1ChangePercent']

0.8674224701075596

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [51]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [52]:
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data['AAPL']['stats'])
#     print(symbol_string.split(','))
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        
        index = my_columns),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,131.20,0.827386,N/A
1,AAL,22.26,1.08392,N/A
2,AAP,211.81,0.770177,N/A
3,AAPL,128.61,0.852534,N/A
4,ABBV,113.90,0.462215,N/A
...,...,...,...,...
500,YUM,121.92,0.475089,N/A
501,ZBH,178.00,0.552019,N/A
502,ZBRA,493.07,1.14392,N/A
503,ZION,60.13,0.881699,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [53]:
final_dataframe.sort_values('One-Year Price Return',ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.96,4.71523,N/A
1,179,FCX,40.85,3.74995,N/A
2,208,GPS,35.69,3.66526,N/A
3,253,IVZ,28.79,2.6605,N/A
4,272,KSS,63.31,2.61514,N/A
5,441,TPR,50.00,2.30863,N/A
6,314,MOS,35.73,2.21116,N/A
7,410,SIVB,582.34,2.10898,N/A
8,23,ALB,168.66,2.02364,N/A
9,24,ALGN,588.29,2.01784,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [54]:
def portfolio_size():
    global portfolio_size
    
    
#     try:
#         float(portfolio_size)
#     except:
#         print('That is not a number! ')
#         print('Please try again:')
#         portfolio_size = input('Enter the size of your portfolio: ')
        
    while True:
        try:
            portfolio_size = float(input('Enter the size of your portfolio: '))
            break
        except ValueError:
            print("Enter a correct value!")
            pass

    #print '\nIncorrect input, try again'
        
portfolio_size()
print(portfolio_size)

final_dataframe

Enter the size of your portfolio: 1000000
1000000.0


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.96,4.71523,N/A
1,179,FCX,40.85,3.74995,N/A
2,208,GPS,35.69,3.66526,N/A
3,253,IVZ,28.79,2.6605,N/A
4,272,KSS,63.31,2.61514,N/A
5,441,TPR,50.00,2.30863,N/A
6,314,MOS,35.73,2.21116,N/A
7,410,SIVB,582.34,2.10898,N/A
8,23,ALB,168.66,2.02364,N/A
9,24,ALGN,588.29,2.01784,N/A


In [55]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
    
final_dataframe

C:\Users\neran\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\neran\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Number of shares to buy
0,275,LB,68.96,4.71523,N/A,290.0
1,179,FCX,40.85,3.74995,N/A,489.0
2,208,GPS,35.69,3.66526,N/A,560.0
3,253,IVZ,28.79,2.6605,N/A,694.0
4,272,KSS,63.31,2.61514,N/A,315.0
5,441,TPR,50.00,2.30863,N/A,400.0
6,314,MOS,35.73,2.21116,N/A,559.0
7,410,SIVB,582.34,2.10898,N/A,34.0
8,23,ALB,168.66,2.02364,N/A,118.0
9,24,ALGN,588.29,2.01784,N/A,33.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                'Ticker',
                'Price',
                'Number of Shares to Buy',
                'One-Year Price Return',
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile'             
            
            ],
        
            index = hqm_columns),
            ignore_index = True
        
        )

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: